# Step By Step Pengerjaan (Deadline 3 September 2024)
- Cocokin data byPlaceData.csv dengan scrapetable_wisata.xlsx
- Mengubah konten menjadi bahasa indonesia, dengan mengambil data dari byPlaceData.csv
- Data yang tidak ada di scrapetable_wisata.xlsx langsung di drop saja di byPlaceData.csv (mengurangi storage)

# Data Gathering

In [74]:
import pandas as pd

# dataLamaPath = 'data/new-fix-data.csv'
dataBaruPath = 'data/scrapetable_wisata.xlsx'
dataByPlace = 'data/byPlaceData.csv'

# dfOld = pd.read_csv(dataLamaPath)
dfNew = pd.read_excel(dataBaruPath)
dfByPlace = pd.read_csv(dataByPlace)

In [75]:
dfNew.head()

,business_id,phone_number,name,full_address,latitude,longitude,review_count,rating,timezone,website,...,Sunday,Monday,Tuesday,Wednesday,Thursday,city,verified,state,validasi di jogja dan tetangga,Hidden_gem
0,0x2e7a5d33a5d8f47d:0xe34419734ac3afc7,6.281915e+12,Wisata Kali Pelangi,"Wisata Kali Pelangi, Pobayan, Kranggan, Manisr...",-7.694183,110.486006,64.0,4.4,Asia/Jakarta,NaN,...,7 AM–11 PM,7 AM–11 PM,7 AM–11 PM,7 AM–11 PM,7 AM–11 PM,"Klaten Regency, Central Java, Indonesia",True,Open ⋅ Closes 11 PM,YA,1
1,0x2e7a5df313462acd:0x1913d8cdf41f83e6,NaN,Obyek Wisata BKK kendal sari,"9FCR+99R Obyek Wisata BKK kendal sari, Unnamed...",-7.629011,110.490877,28.0,4.3,Asia/Jakarta,NaN,...,NaN,NaN,NaN,NaN,NaN,"Klaten Regency, Central Java, Indonesia",False,NaN,YA,1
2,0x2e7b01c7c0bc9f83:0xeb7a328ebb940844,NaN,Pantai Parangkusumo,"Pantai Parangkusumo, Parang kusumo, Pantai, Pa...",-8.023053,110.325250,1285.0,4.6,Asia/Jakarta,NaN,...,NaN,NaN,NaN,NaN,NaN,"Bantul Regency, Jawa Tengah, Indonesia",True,NaN,YA,1
3,0x2e7a8cf009a7d697:0xdd34334744dc3cb,6.229379e+10,Borobudur Temple,"Borobudur Temple, Jl. Badrawati, Kw. Candi Bor...",-7.607874,110.203751,94732.0,4.7,Asia/Jakarta,http://borobudurpark.com/temple/borobudur/,...,6:30 AM–4:30 PM,6:30 AM–4:30 PM,6:30 AM–4:30 PM,6:30 AM–4:30 PM,6:30 AM–4:30 PM,"Magelang Regency, Central Java, Indonesia",True,Open ⋅ Closes 4:30 PM,YA,1
4,0x2e7a9be3a62bbdaf:0x3a246be0b4291afa,6.287706e+11,Wisata Negeri Sayur Sukomakmur (Terasering Sit...,Wisata Negeri Sayur Sukomakmur (Terasering Sit...,-7.417720,110.064824,1096.0,4.7,Asia/Jakarta,NaN,...,4:30 AM–5 PM,4:30 AM–5 PM,4:30 AM–5 PM,4:30 AM–5 PM,4:30 AM–5 PM,"Magelang Regency, Central Java, Indonesia",True,Open ⋅ Closes 5 PM,YA,1


In [76]:
dfByPlace.head()

,Unnamed: 0,name,address,coordinates,workday_timing,closed_on,reviews,rating,most_popular_times,popular_times,reviewer_name,rating_review,review_text,published_at_date,accessibility_enabled,planning_enabled,children_enabled,is_rating_updated,is_reviews_updated
0,0,( Alun Alun Utara),"59W8+F66 ( Alun Alun Utara), Jl. Alun-Alun Ut...","-7.8038427,110.3655155",Buka 24 jam,Open All Days,43928,4.8,Not Present,Not Present,"['fandy achmad hamid', 'Lucky Zacky']","[4.0, 5.0]",['Tempat Nongkrong yang enak dikunjungi bareng...,"['2023-07-03 18:57:13.071', '2023-11-07 08:23:...","Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak,1.0,1.0
1,1,Embung Kladuan UII,"Embung Kladuan UII, Moh. Hatta, Lodadi, Umbul...","-7.6889138,110.4159846",Buka 24 jam,Open All Days,5101,4.7,"08.00, 09.00, 11.00\n---\nTime Label: 08.00\nA...",Monday:\n 04.00: 50% | Biasanya tidak terla...,"['Rizky D. Novyantika', 'Rahajeng Anindyajati'...","[4.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0]","['Embung ini sebenernya lumayan buat nyore, ta...","['2022-10-05 10:41:56.075', '2022-01-01 10:55:...","Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak,1.0,1.0
2,2,Jembatan bok bang,"6PH9+4W8 Jembatan bok bang, Unnamed Road, Are...","-7.7722092,110.7198427",Buka 24 jam,Open All Days,17765,4.6,"23.00, 22.00, 00.00\n---\nTime Label: 23.00\nA...",Monday:\n 04.00: 0% | Idle\n 05.00: 0% |...,['Boneka Mainan'],[5.0],"['Saat saya memancing, saya sangat senang. Pa...",['2021-02-19 11:02:46.654'],Tempat parkir khusus pengguna kursi roda,Tidak menerima reservasi,Tidak cocok untuk anak-anak,1.0,1.0
3,3,Sendang Sari Mulyo,"2CMC+GCH Sendang Sari Mulyo, Unnamed Road, No...","-7.9661915,110.4211151",08.00-16.00,Open All Days,12,4.9,Not Present,Not Present,"['Muhammad Zaidan Robbani', 'Handika Roma', 'F...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0]",['Ada sentra pembuatan bawang goreng jugaaa lo...,"['2021-07-03 05:39:15.538', '2022-11-02 02:04:...","Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak,0.0,0.0
4,4,Stupa Buddha Dawangsari,"6FGX+926 Stupa Buddha Dawangsari, Dawang Sari...","-7.7740887999999995,110.4975261",08.00-16.00,Open All Days,16,4.9,Not Present,Not Present,"['Keperluan Penelitian', 'Dini Suryaningsih', ...","[5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 5.0, 5.0]",['Candi Dawangsari merupakan peninggalan Stupa...,"['2023-10-29 00:06:11.689', '2023-05-24 12:39:...",Tempat parkir khusus pengguna kursi roda,Tidak menerima reservasi,Tidak cocok untuk anak-anak,1.0,1.0


# Data Assessing

## Cek Info DF

In [80]:
# ByPlace
print('By Place')
print(dfByPlace.info())

print()

# new
print('New')
print(dfNew.info())

By Place
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13992 entries, 0 to 13991
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             13992 non-null  int64  
 1   name                   13992 non-null  object 
 2   address                13992 non-null  object 
 3   coordinates            13992 non-null  object 
 4   workday_timing         13992 non-null  object 
 5   closed_on              13992 non-null  object 
 6   reviews                13992 non-null  int64  
 7   rating                 13992 non-null  float64
 8   most_popular_times     13992 non-null  object 
 9   popular_times          13992 non-null  object 
 10  reviewer_name          13992 non-null  object 
 11  rating_review          13992 non-null  object 
 12  review_text            13992 non-null  object 
 13  published_at_date      13992 non-null  object 
 14  accessibility_enabled  13992 non-null  object

## Cek Nilai Null

In [81]:
# ByPlace
print('By Place')
print(dfByPlace.isna().sum())

print()

# new
print('New')
print(dfNew.isna().sum())

By Place
Unnamed: 0                   0
name                         0
address                      0
coordinates                  0
workday_timing               0
closed_on                    0
reviews                      0
rating                       0
most_popular_times           0
popular_times                0
reviewer_name                0
rating_review                0
review_text                  0
published_at_date            0
accessibility_enabled        0
planning_enabled             0
children_enabled             0
is_rating_updated        13881
is_reviews_updated       13881
dtype: int64

New
business_id                         0
phone_number                      308
name                                0
full_address                        0
latitude                            0
longitude                           0
review_count                       84
rating                             84
timezone                            0
website                           461
plac

## Cek Data Duplikat

In [82]:
# ByPlace
print('By Place')
print(dfByPlace.duplicated().sum())

print()

# new
print('New')
print(dfNew.duplicated().sum())

By Place
0

New
0


# Data Cleaning

## Feature Selection